In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv')
train_100_df = train_df.head(100)
train_100_df.to_csv('train_100.csv', index=False)


In [2]:
file_path = 'train_100.csv'
df = pd.read_csv(file_path)
df.drop('id', axis=1, inplace=True)

In [3]:
df.head()

,conversations
0,"[{'from': 'human', 'value': ""I've been feeling..."
1,"[{'from': 'human', 'value': ""Hi, I'm feeling r..."
2,"[{'from': 'human', 'value': ""Hey, I hope you'r..."
3,"[{'from': 'human', 'value': ""I'm feeling reall..."
4,"[{'from': 'human', 'value': ""I'm feeling reall..."


In [4]:
first_row = df.head(1)
print(first_row)

                                       conversations
0  [{'from': 'human', 'value': "I've been feeling...


In [5]:
columns_to_replace = ['conversations']

for column in columns_to_replace:
    df[column] = df[column].str.replace('from', 'role', case=False)
    df[column] = df[column].str.replace('human', 'user', case=False)
    df[column] = df[column].str.replace('value', 'content', case=False)
    df[column] = df[column].str.replace('gpt', 'assistant', case=False)


In [6]:
first_row = df.head(1)
print(first_row)

                                       conversations
0  [{'role': 'user', 'content': "I've been feelin...


In [7]:
output_file_path = 'modified_train_100.csv'
df.to_csv(output_file_path, index=False)

In [10]:
df['conversations'] = df['conversations'].str.replace('\n', ' ')

In [11]:
first_row = df.head(1)
print(first_row)

                                       conversations
0  [{'role': 'user', 'content': "I've been feelin...


In [33]:
df.rename(columns={'conversations': 'messages'}, inplace=True)
output_file_path = 'modified_train_100.csv'
df.to_csv(output_file_path, index=False)

In [34]:
jsonl_file_path = 'output.jsonl'

df.to_json(jsonl_file_path, orient='records', lines=True)

##output.jsonl still need more cleaning before passing to openai, did it manually for now

In [8]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

client.files.create(
  file=open("output.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-FU1xJsJpMrNbTYrPLhVXzfnv', bytes=136705, created_at=1706016635, filename='output.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
  training_file="file-FU1xJsJpMrNbTYrPLhVXzfnv", 
  model="gpt-3.5-turbo"
)

In [ ]:
client.fine_tuning.jobs.retrieve("ftjob-XSK09YKVFJKGgF8IeSrc2nhJ")

In [25]:
from openai import OpenAI
import os
client = OpenAI()

response = client.chat.completions.create(
  model=os.getenv("FT_MODEL"),
  messages=[
    {"role": "system", "content": "You are a mental health therapist"},
    {"role": "user", "content": "I feel so shocked and overwhelmed by this constant weight on my chest."}
  ]
)
print(response.choices[0].message.content)

It sounds like you're carrying a heavy burden. Can you tell me more about what's been going on?
